In [ ]:
#Based on Paulo Pinto's Kernel: 0.529 on Public Leaderboard
#https://www.kaggle.com/paulorzp/log-ma-and-days-of-week-means-lb-0-529/code

In [1]:
#### Load data and preprocessing

import pandas as pd
import numpy as np
from datetime import timedelta

#Train colums: id, date, store_nbr, item_nbr, unit_sales, onpromotion

dtypes = {'id':'uint32', 'item_nbr':'int32', 'store_nbr':'int8', 'unit_sales':'float32'}
train_orig = pd.read_csv('data/train.csv', usecols=[1,2,3,4,5], #ignore 'id'
                    dtype=dtypes, parse_dates=['date'],
                    skiprows=range(1, 83657331)) #Skip dates before 2016-07-01

# add day of week as a new feature
train_orig['dow'] = train_orig['date'].dt.dayofweek

# drop rows of returned items
train_orig = train_orig[train_orig.unit_sales > 0]

# convert sales to log
train_orig['unit_sales'] =  train_orig['unit_sales'].apply(np.log1p)
train_orig.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41836954 entries, 0 to 41839709
Data columns (total 6 columns):
date           datetime64[ns]
store_nbr      int8
item_nbr       int32
unit_sales     float32
onpromotion    bool
dow            int64
dtypes: bool(1), datetime64[ns](1), float32(1), int32(1), int64(1), int8(1)
memory usage: 1.3 GB


In [3]:
print(train_orig.head())

        date  store_nbr  item_nbr  unit_sales  onpromotion  dow
0 2016-06-30         54   2027252    1.098612        False    3
1 2016-07-01          1    103520    0.693147        False    4
2 2016-07-01          1    103665    1.098612        False    4
3 2016-07-01          1    105574    2.197225        False    4
4 2016-07-01          1    105575    2.708050        False    4


In [10]:
#### Feature engineering 1 : baseline predictor
#### Average of daily sales Using recent training data

train = train_orig.drop('dow', axis=1)

# Transformation of the training data.
# creating records for all items, in all stores on all dates
# for calculations of daily sales averages for an item in each store.
u_dates  = train.date.unique()
u_stores = train.store_nbr.unique()
u_items  = train.item_nbr.unique()
train.set_index(['date', 'store_nbr', 'item_nbr'], inplace=True)
train = train.reindex(
    pd.MultiIndex.from_product(
        (u_dates, u_stores, u_items)
        ,names=['date','store_nbr','item_nbr']
    )
).reset_index()

train.loc[:, 'unit_sales'].fillna(0, inplace=True) # fill NaNs
lastdate = train.iloc[train.shape[0]-1].date
print(train.head())
del u_dates, u_stores, u_items

## Engineering features
# Use all the data from training set (one year of data)
sales_daily_avg = train[['item_nbr','store_nbr','unit_sales']].groupby(
        ['item_nbr','store_nbr'])['unit_sales'].mean().to_frame('sales_daily_avg')

# More predictors
# Use only the most recent data from the training set, unit: day
# Use the median of all predictors as the final predictor.
for i in [112,56,28,14,7,3,1]: #days are relatively arbituray
    tmp = train[train.date>lastdate-timedelta(int(i))]
    tmpg = tmp.groupby(['item_nbr','store_nbr'])['unit_sales'].mean().to_frame('sales_daily_avg_'+str(i))
    sales_daily_avg = sales_daily_avg.join(tmpg, how='left')

sales_daily_avg['sales_daily_avg']=sales_daily_avg.median(axis=1)
sales_daily_avg.reset_index(inplace=True) #reset index for merging later
sales_daily_avg.drop(list(sales_daily_avg.columns.values)[3:],axis=1,inplace=True)
sales_daily_avg.head()

        date  store_nbr  item_nbr  unit_sales onpromotion
0 2016-06-30         54   2027252    1.098612       False
1 2016-06-30         54    103520    0.000000         NaN
2 2016-06-30         54    103665    0.000000         NaN
3 2016-06-30         54    105574    0.000000         NaN
4 2016-06-30         54    105575    0.000000         NaN


,item_nbr,store_nbr,sales_daily_avg
0,96995,1,0.126638
1,96995,2,0.024755
2,96995,3,0.355917
3,96995,4,0.124828
4,96995,5,0.118639


In [7]:
#### Feature engineering 2: modifiers for all items
#### Weekday effect: different sales on weekdays and weekends
# Weekday_avg
# Reference: By tarobxl: https://www.kaggle.com/c/favorita-grocery-sales-forecasting/discussion/42948

# For a particular store, a specific item, how many on average are sold on a Monday/Tuesday/... ?
sales_weekday_avg = train_orig[['item_nbr','store_nbr','dow','unit_sales']].groupby(
        ['store_nbr','item_nbr','dow'])['unit_sales'].mean().to_frame('sales_weekday_avg').reset_index()
print(sales_weekday_avg.head())

# Week_avg
#From the calculations above, average over all 7 days.
sales_week_avg = sales_weekday_avg[['item_nbr','store_nbr','sales_weekday_avg']].groupby(
        ['store_nbr', 'item_nbr'])['sales_weekday_avg'].mean().to_frame('sales_week_avg').reset_index()
print(sales_week_avg.info())
print(sales_week_avg.head())

print(sales_weekday_avg.info())
print(sales_weekday_avg.head())

   store_nbr  item_nbr  dow  sales_weekday_avg
0          1     96995    0           0.895880
1          1     96995    1           0.693147
2          1     96995    2           0.693147
3          1     96995    3           0.794513
4          1     96995    4           0.943827
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171118 entries, 0 to 171117
Data columns (total 3 columns):
store_nbr         171118 non-null int64
item_nbr          171118 non-null int64
sales_week_avg    171118 non-null float32
dtypes: float32(1), int64(2)
memory usage: 3.3 MB
None
   store_nbr  item_nbr  sales_week_avg
0          1     96995        0.844024
1          1     99197        1.010969
2          1    103520        1.100751
3          1    103665        1.325922
4          1    105574        1.689430
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1182588 entries, 0 to 1182587
Data columns (total 4 columns):
store_nbr            1182588 non-null int64
item_nbr             1182588 non-null int

In [22]:
#Load test
test = pd.read_csv('data/test.csv', dtype=dtypes, parse_dates=['date'])
test['dow'] = test['date'].dt.dayofweek
print(test.head())
test = pd.merge(test, sales_daily_avg,   how='left', on=['item_nbr','store_nbr'])
test = pd.merge(test, sales_week_avg,    how='left', on=['item_nbr','store_nbr'])
test = pd.merge(test, sales_weekday_avg, how='left', on=['item_nbr','store_nbr','dow'])
print(test.head())

          id       date  store_nbr  item_nbr  onpromotion  dow
0  125497040 2017-08-16          1     96995        False    2
1  125497041 2017-08-16          1     99197        False    2
2  125497042 2017-08-16          1    103501        False    2
3  125497043 2017-08-16          1    103520        False    2
4  125497044 2017-08-16          1    103665        False    2
          id       date  store_nbr  item_nbr  onpromotion  dow  \
0  125497040 2017-08-16          1     96995        False    2   
1  125497041 2017-08-16          1     99197        False    2   
2  125497042 2017-08-16          1    103501        False    2   
3  125497043 2017-08-16          1    103520        False    2   
4  125497044 2017-08-16          1    103665        False    2   

   sales_daily_avg  sales_week_avg  sales_weekday_avg  
0         0.126638        0.844024           0.693147  
1         0.198430        1.010969           0.977616  
2         0.000000             NaN                NaN  
3

In [23]:
#Forecasting Test
test['unit_sales'] = test.sales_daily_avg

pos_idx  = test['sales_week_avg'] > 0
test_pos = test.loc[pos_idx]

# Add weekday-effect to the daily averge sales
weekday_scale = test_pos['sales_weekday_avg'] / test_pos['sales_week_avg']
test.loc[pos_idx, 'unit_sales'] = test_pos['sales_daily_avg'] * weekday_scale

test.loc[:, "unit_sales"].fillna(0, inplace=True)

#test['unit_sales_final'] = test['unit_sales']

# a 150% multiplier for Promoted items 
id_promo = test.onpromotion == True
test.loc[id_promo, 'unit_sales'] = np.log1p(test.loc[id_promo, 'unit_sales'].apply(np.expm1) * \
                                                  1.5
#                                                  test.loc[id_promo, 'promo_rate']
                                                 )
test.head()

,id,date,store_nbr,item_nbr,onpromotion,dow,sales_daily_avg,sales_week_avg,sales_weekday_avg,unit_sales
0,125497040,2017-08-16,1,96995,False,2,0.126638,0.844024,0.693147,0.104000
1,125497041,2017-08-16,1,99197,False,2,0.198430,1.010969,0.977616,0.191884
2,125497042,2017-08-16,1,103501,False,2,0.000000,NaN,NaN,0.000000
3,125497043,2017-08-16,1,103520,False,2,0.649946,1.100751,1.152717,0.680630
4,125497044,2017-08-16,1,103665,False,2,1.006661,1.325922,1.313600,0.997306


In [17]:
test['unit_sales'] = test['unit_sales_final'].apply(pd.np.expm1) # restoring unit values 
# write submission file
test[['id','unit_sales']].to_csv('submissions/ma8dwof_reproduce.csv.gz', index=False, float_format='%.3f', compression='gzip')
print('Done!')

Done!
